In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

class DC_WGAN(object):
    def __init__(self, mb_size, X_dim, z_dim, h_dim, penalty=10, n_disc=5):
        self.mb_size = mb_size
        self.X_dim = X_dim
        self.z_dim = z_dim
        self.h_dim = h_dim
        self.penalty = penalty
        self.n_disc = 5 ## number of disciminator steps per generator step
        
    def lrelu(self, x, th=0.1):
        return tf.maximum(th * x, x)    
    
    def sample_z(self, m):
        return np.random.normal(0, 1, (m, 1, 1, 100))
    
    def plot(self, samples):
        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(64, 64), cmap='Greys_r')

        return fig

    def generator(self, x, isTrain=True, reuse=False):
        with tf.variable_scope('generator', reuse=reuse):

            # 1st hidden layer
            conv1 = tf.layers.conv2d_transpose(x, 1024, [4, 4], strides=(1, 1), padding='valid')
            lrelu1 = self.lrelu(tf.layers.batch_normalization(conv1, training=isTrain), 0.2)

            # 2nd hidden layer
            conv2 = tf.layers.conv2d_transpose(lrelu1, 512, [4, 4], strides=(2, 2), padding='same')
            lrelu2 = self.lrelu(tf.layers.batch_normalization(conv2, training=isTrain), 0.2)

            # 3rd hidden layer
            conv3 = tf.layers.conv2d_transpose(lrelu2, 256, [4, 4], strides=(2, 2), padding='same')
            lrelu3 = self.lrelu(tf.layers.batch_normalization(conv3, training=isTrain), 0.2)

            # 4th hidden layer
            conv4 = tf.layers.conv2d_transpose(lrelu3, 128, [4, 4], strides=(2, 2), padding='same')
            lrelu4 = self.lrelu(tf.layers.batch_normalization(conv4, training=isTrain), 0.2)

            # output layer
            conv5 = tf.layers.conv2d_transpose(lrelu4, 1, [4, 4], strides=(2, 2), padding='same')
            o = tf.nn.tanh(conv5)

            return o

    def discriminator(self, x, isTrain=True, reuse=False):
        with tf.variable_scope('discriminator', reuse=reuse):
            # 1st hidden layer
            conv1 = tf.layers.conv2d(x, 128, [4, 4], strides=(2, 2), padding='same')
            lrelu1 = self.lrelu(conv1, 0.2)

            # 2nd hidden layer
            conv2 = tf.layers.conv2d(lrelu1, 256, [4, 4], strides=(2, 2), padding='same')
            lrelu2 = self.lrelu(tf.layers.batch_normalization(conv2, training=isTrain), 0.2)

            # 3rd hidden layer
            conv3 = tf.layers.conv2d(lrelu2, 512, [4, 4], strides=(2, 2), padding='same')
            lrelu3 = self.lrelu(tf.layers.batch_normalization(conv3, training=isTrain), 0.2)

            # 4th hidden layer
            conv4 = tf.layers.conv2d(lrelu3, 1024, [4, 4], strides=(2, 2), padding='same')
            lrelu4 = self.lrelu(tf.layers.batch_normalization(conv4, training=isTrain), 0.2)

            # output layer
            conv5 = tf.layers.conv2d(lrelu4, 1, [4, 4], strides=(1, 1), padding='valid')
            o = tf.nn.sigmoid(conv5)

            return o
        
        
    def training(self):
        tf.reset_default_graph()
        with tf.name_scope('inputs'):
            X = tf.placeholder(tf.float32, shape=(None, 64, 64, 1)) ## dataset
            z = tf.placeholder(tf.float32, shape=(None, 1, 1, 100)) ## noise passed to generator
        
        G_sample = self.generator(z)
        D_real = self.discriminator(X)
        D_fake = self.discriminator(G_sample, reuse=True)

        D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
        G_loss = -tf.reduce_mean(D_fake)

        alpha = tf.random_uniform(shape=[self.mb_size,64, 64, 1], minval=0.,maxval=1.)
        differences = D_fake - D_real
#         print(differences.shape)
        interpolates = D_real + (alpha*differences)
        gradients = tf.gradients(self.discriminator(interpolates, reuse=True), [interpolates])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1.)**2)
        D_loss += self.penalty*gradient_penalty


        T_vars = tf.trainable_variables()
        D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
        G_vars = [var for var in T_vars if var.name.startswith('generator')]

        G_solver = tf.train.AdamOptimizer(learning_rate=1e-1, beta1=0.5,
                                          beta2=0.9).minimize(G_loss, var_list=G_vars)


        D_solver = tf.train.AdamOptimizer(learning_rate=1e-1, beta1=0.5, 
                                          beta2=0.9).minimize(D_loss, var_list=D_vars)

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        if not os.path.exists('out_improvedGAN_dgscan/'):
            os.makedirs('out_improvedGAN_dgscan/')

        i = 0
        
        # MNIST resize and normalization
        mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, reshape=[])
        train_set = tf.image.resize_images(mnist.train.images, [64, 64]).eval(session=sess)
        train_set = (train_set - 0.5) / 0.5  # normalization; range: -1 ~ 1

        for it in range(10000):
            
            for _ in range(self.n_disc): 
                X_mb = train_set[it*mb_size:(it+1)*mb_size]
                _, D_loss_curr = sess.run([D_solver, D_loss],
                    feed_dict={X: X_mb, z: self.sample_z(mb_size)})

            _, G_loss_curr = sess.run([G_solver, G_loss],
                feed_dict={z: self.sample_z(mb_size)})

            if it % 1 == 0:
                print('Iter: {}; D loss: {:.4}; G_loss: {:.4}'.format(it, D_loss_curr, G_loss_curr))
                samples = sess.run(G_sample, feed_dict={z: self.sample_z(16)})

                fig = self.plot(samples)
                plt.savefig('out_improved_DCGAN/{}.png'
                            .format(str(i).zfill(3)), bbox_inches='tight')
                i += 1
                plt.close(fig) 

In [6]:
mb_size = 16
X_dim = 4096
z_dim = 10
h_dim = 64
n_disc = 5

mnist_train = DC_WGAN(mb_size, X_dim, z_dim, h_dim)

In [7]:
mnist_train.training()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0; D loss: 9.181; G_loss: -0.01555
Iter: 1; D loss: 0.9266; G_loss: -5.441e-05
Iter: 2; D loss: 10.0; G_loss: -2.811e-05


KeyboardInterrupt: 